## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [1]:
#.\.venv\Scripts\activate

import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In this example, we will use the checkpoints from OpenVoiceV2. OpenVoiceV2 is trained with more aggressive augmentations and thus demonstrate better robustness in some cases.

In [2]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

c:\Users\kimyk\OneDrive\문서\유경\OpenVoice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\Users\kimyk\OneDrive\문서\유경\OpenVoice\.venv\lib\site-packages\wavmark\__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_glo

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding
We only extract the tone color embedding for the target speaker. The source tone color embeddings can be directly loaded from `checkpoints_v2/ses` folder.

In [ ]:
import time
from speech_recognition import Recognizer, Microphone
from pydub import AudioSegment

src_path = f'resources/tmp_microphone.mp3'

recognizer = Recognizer()

duration_seconds = 30  # 녹음 시간 (초)
chunk_duration = 5     # 청크별 녹음 시간 (초)
audio_segments = []    # 녹음된 데이터 저장 리스트


with Microphone() as source:
    print("녹음 시작: 말을 시작하세요")
    recognizer.adjust_for_ambient_noise(source)  # 주변 소음 보정
    start_time = time.time()

    while time.time() - start_time < duration_seconds:
        print(f"{chunk_duration}초 동안 녹음 중")
        audio_data = recognizer.listen(source, timeout=chunk_duration, phrase_time_limit=chunk_duration)
        audio_segments.append(audio_data.get_wav_data())
        print(f"청크 녹음 완료: 현재 녹음 시간 {int(time.time() - start_time)}초")


# WAV 파일로 저장
print("녹음 완료: 저장 중")
final_audio = AudioSegment.empty()
for segment in audio_segments:
    audio_chunk = AudioSegment(segment)
    final_audio += audio_chunk

# 파일 저장
final_audio.export(src_path, format="mp3")
print(f"녹음된 파일 저장 완료: {src_path}")


녹음 시작: 말을 시작하세요
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 5초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 8초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 13초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 14초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 19초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 24초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 26초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 31초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 36초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 38초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 43초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 45초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 47초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 49초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 51초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 56초
5초 동안 녹음 중...
청크 녹음 완료: 현재 녹음 시간 60초
녹음 완료: 저장 중...
녹음된 파일 저장 완료: resources/tmp_microphone.mp3


In [ ]:

reference_speaker = src_path  # This is the voice you want to clone

'''
음성에서 음색 특성과 관련 정보를 추출
    target_se: 입력 음성에서 추출된 음색 특성 (벡터)
    audio_name: 입력된 음성 파일의 이름
'''
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2
[(0.0, 8.21), (9.39, 12.946), (13.262, 29.49225)]
after vad: dur = 27.995986394557825


#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

In [ ]:
from melo.api import TTS

# 변환할 텍스트 데이터 정의
text = "안녕하세요! 오늘은 날씨가 정말 좋네요."

# 텍스트를 음성으로 변환한 결과를 저장하기 위한 임시 파일 경로
src_path = f'{output_dir}/tmp.wav'

# Speed is adjustable
speed = 1.0

model = TTS(language='KR', device=device)  
speaker_ids = model.hps.data.spk2id

for speaker_key in speaker_ids.keys():
    speaker_id = speaker_ids[speaker_key]
    speaker_key = speaker_key.lower().replace('_', '-')
    
    source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
    model.tts_to_file(text, speaker_id, src_path, speed=speed)
    save_path = f'{output_dir}/output_v2_microphone.wav'

    # Run the tone color converter (생성된 음성의 음색을 변환)
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path, 
        src_se=source_se, 
        tgt_se=target_se, 
        output_path=save_path,
        message=encode_message)

c:\Users\kimyk\OneDrive\문서\유경\OpenVoice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\kimyk\OneDrive\문서\유경\OpenVoice\MeloTTS\melo\download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

 > Text split to sentences.
안녕하세요! 오늘은 날씨가 정말 좋네요.
 > ===========================


100%|██████████| 1/1 [00:32<00:00, 32.60s/it]
